## TFL Images
* 15/3/2020
* Notebook to download, classify and produce chart data for TFL traffic cams

In [2]:
from pathlib import Path
import requests
from PIL import Image
from io import BytesIO
from datetime import datetime, timedelta
from fastai.vision import load_learner, open_image
import json

### Setup
Create file data and file paths

In [3]:
path = Path('/media/jjc/data/dataStore/fastai_projects/carparks/')
path

PosixPath('/media/jjc/data/dataStore/fastai_projects/carparks')

In [4]:
run_date = datetime(2020,3,9)


In [6]:
# create a datetime for every 5 minutes in the day, corresponding to the TFL update frequency
time_list = [run_date + timedelta(minutes= 5*x) for x in range(288)]

In [7]:
time_list[-3:]

[datetime.datetime(2020, 3, 9, 23, 45),
 datetime.datetime(2020, 3, 9, 23, 50),
 datetime.datetime(2020, 3, 9, 23, 55)]

In [8]:
# create directories if not present
tfl_images = path / 'tfl_images'
date_str = run_date.strftime('%Y-%m-%d')
date_images = tfl_images / date_str

In [9]:
date_images.mkdir(exist_ok=True)
date_images

PosixPath('/media/jjc/data/dataStore/fastai_projects/carparks/tfl_images/2020-03-09')

In [14]:
url_template = 'http://archive.tfljamcams.net/archive/Gt_Eastern_St_Curtain_Rd/{date}/{file}'

In [15]:
# list to record results in
success_file=[]

# loop through the time slots for a given day
for t in time_list:
    file_str = t.strftime('%Y-%m-%d@%H_%MGMT.jpg')
    
    file_path = date_images / file_str
    if file_path.exists(): # already downloaded
        success_file.append({'file_name':file_str, 'file_time':t})
    else:
        try:
            # download and save images, handling any data corruption errors
            url = url_template.format(date=date_str, file=file_str)
            r = requests.get(url)
            i = Image.open(BytesIO(r.content))
            i.save(file_path)
            success_file.append({'file_name':file_str, 'file_time':t})
        except OSError:
            print('error ',file_str)

error  2020-03-09@14_40GMT.jpg
error  2020-03-09@21_10GMT.jpg


### Load model and perform classification

In [16]:
model = load_learner(path).to_fp16() # load model that was previously exported

In [17]:
out_json=[]

# loop through the images and have the model classify them
for file in success_file:
    image = open_image(date_images / file['file_name']).apply_tfms(None, size=224)

    category, class_idx ,prob = model.predict(image)
    
    out_json.append({'time':file['file_time'],
                     'category':category.obj,
                     'category_class':class_idx.item(),
                     'prob':prob[class_idx].item()})

In [18]:
# see results
out_json[:3]

[{'time': datetime.datetime(2020, 3, 9, 0, 0),
  'category': 'full',
  'category_class': 1,
  'prob': 0.9771101474761963},
 {'time': datetime.datetime(2020, 3, 9, 0, 5),
  'category': 'full',
  'category_class': 1,
  'prob': 0.9771101474761963},
 {'time': datetime.datetime(2020, 3, 9, 0, 10),
  'category': 'full',
  'category_class': 1,
  'prob': 0.9829874038696289}]

In [19]:
def line_chart_js(data:dict,label:str):
    """Function to turn a given dictionary into the json format used by chart.js"""
    
    # time boundaries for the axis
    time_min = min([d['time'] for d in data])
    time_max = max([d['time'] for d in data])
    
    # prepare base json data
    json_returned = {
    'data':{'labels':['class','prob'],'datasets':[]},
    'options': {'title': {'display': True, 'text': label},
                'tooltips': {'mode': 'index','intersect': False},
                'responsive': True,
                'scales': {'xAxes': [{'type': 'time',
                                      'time':{
                                        'unit': 'minute',
                                        'displayFormats': { 'month': 'mm' },
                                        'max': time_max.strftime("%Y-%m-%d %H:%M"),
                                        'min': time_min.strftime("%Y-%m-%d %H:%M"),
                                        },
                                      'scaleLabel': {'display': True,'labelString': 'Time'}}],
                           'yAxes': [{'scaleLabel': {'display': True,'labelString': 'Value'}}]}
               }
    }

    # populate the data
    line_class_data = [{'x':d['time'].strftime("%Y-%m-%d %H:%M") ,'y':d['category_class']} for d in data]
    line_prob_data = [{'x':d['time'].strftime("%Y-%m-%d %H:%M") ,'y':d['prob']} for d in data]

    # add the data to the json object
    json_returned['data']['datasets'].append({'label':'class',
                                              'borderColor': 'blue',
                                              'fill':True,
                                              'type':'line',
                                              'data':line_class_data})
    json_returned['data']['datasets'].append({'label':'prob',
                                              'borderColor': 'red',
                                              'fill':True,
                                              'type':'line',
                                              'data':line_prob_data})

    return json_returned


In [20]:
json_data = line_chart_js(out_json,date_str)

In [22]:
# save the data down
with open(path / f'traffic_{date_str}.json','w') as fp:
    json.dump(json_data,fp)